<a href="https://colab.research.google.com/github/shirooo39/YASDCN/blob/main/YASDCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font size=8><center>**Welcome to <a href="https://github.com/shirooo39/YASDCN" target="_blank">YASDCN</a>**</center></font>

YASDCN or short for Yet Another Stable Diffusion Colab Notebook  
This notebook is just a stripped down version of [my MiXLab notebook](https://colab.research.google.com/github/shirooo39/MiXLab/blob/master/MiXLab.ipynb)

[![ko-fi](https://img.shields.io/badge/Support%20me%20on%20Ko--fi-F16061?logo=ko-fi&logoColor=white&style=flat)](https://ko-fi.com/shiro39)

# <font size=6>✦ Mount / Unmount Drives ✦</font>

## <img src="https://raw.githubusercontent.com/shirooo39/MiXLab/master/resources/images/google_drive.png" height="45" align="left"><font size="6">Google Drive</font>

In [ ]:
import pathlib
import shutil
import time
from os import chdir
from google.colab import drive
from IPython.display import HTML, clear_output


def print_html(text='', color=''):
    display(HTML(f'<p style="font-size: 18px; font-weight: bold; color: {color};">{text}</p>'))


def main():
    #@markdown <font size='5'>← Mount / Unmount Google Drive</font>
    action = 'mount' #@param ['mount', 'unmount', 'force_remount']
    mount_path = '/content/drive/google'

    chdir('/content')

    if action == 'mount':
        try:
            clear_output()

            print_html('Mounting Google Drive...')
            time.sleep(5)

            if not pathlib.Path(mount_path).exists():
                pathlib.Path(mount_path).mkdir(parents=True, exist_ok=True)

            drive.mount(mount_path)
        except:
            clear_output()
            print_html('❌ Unable to mount Google Drive', '#F44336')
        else:
            clear_output()
            print_html(f'✅ Google Drive has been mounted to {mount_path}', '#4CAF50')
    elif action == 'unmount':
        try:
            drive.flush_and_unmount()
        except:
            clear_output()
            print_html('❌ Unable to unmount Google Drive', '#F44336')
        else:
            if pathlib.Path('/root/.config/Google/DriveFS').exists() and pathlib.Path('/root/.config/Google/DriveFS').is_dir():
                shutil.rmtree('/root/.config/Google/DriveFS')

            clear_output()
            print_html('✅ Google Drive has been unmounted', '#4CAF50')
    elif action == 'force_remount':
        try:
            clear_output()

            print_html('Mounting Google Drive...')
            time.sleep(5)

            if not pathlib.Path(mount_path).exists():
                pathlib.Path(mount_path).mkdir(parents=True, exist_ok=True)

            drive.mount(mount_path, force_remount=True)
        except:
            clear_output()
            print_html('❌ Unable to mount Google Drive', '#F44336')
        else:
            clear_output()
            print_html('✅ Google Drive has been mounted', '#4CAF50')


if __name__ == '__main__':
    main()


## <img src="https://raw.githubusercontent.com/shirooo39/MiXLab/master/resources/images/rclone.png" height="45" align="left"><font size="6">rclone VFS Drive</font>

In [ ]:
import subprocess
import time
from os import chdir
from IPython.display import HTML, clear_output


def print_html(text='', color=''):
    display(HTML(f'<p style="font-size: 18px; font-weight: bold; color: {color};">{text}</p>'))


def main():
    #@markdown <font size='5'>← Install rclone</font>
    version = 'stable' #@param ['stable', 'beta']

    chdir('/content')

    print_html('Installing rclone...')
    time.sleep(5)

    if version == 'stable':
        subprocess.run('curl https://rclone.org/install.sh | sudo bash', shell=True)
    elif version == 'beta':
        subprocess.run('curl https://rclone.org/install.sh | sudo bash -s beta', shell=True)
    
    subprocess.run('apt-get install -y fuse3', shell=True)

    try:
        subprocess.run('rclone --version', shell=True, check=True)
    except:
        clear_output()
        print_html('❌ Unable to install rclone', '#F44336')
    else:
        clear_output()
        print_html('✅ rclone has been installed', '#4CAF50')


if __name__ == '__main__':
    main()


In [ ]:
import pathlib
import shutil
import time
from os import chdir
from google.colab import files
from IPython.display import HTML, clear_output


def print_html(text='', color=''):
    display(HTML(f'<p style="font-size: 18px; font-weight: bold; color: {color};">{text}</p>'))


def main():
    #@markdown <font size='5'>← Upload rclone.conf</font>
    upload_path = pathlib.Path('/content')

    chdir('/content')

    print_html('Upload rclone.conf from your local machine...')
    time.sleep(5)
    uploaded_file = files.upload()

    for filename in uploaded_file.keys():
        if filename != 'rclone.conf':
            pathlib.Path(filename).unlink()

            clear_output()
            print_html('❌ Please upload only rclone.conf', '#F44336')
        else:
            source_path = pathlib.Path(filename)
            destination_path = upload_path / filename

            shutil.move(source_path, destination_path)

            clear_output()
            print_html('✅ rclone.conf has been uploaded', '#4CAF50')


if __name__ == '__main__':
    main()


In [ ]:
import os
import pathlib
import re
import shlex
import subprocess
import uuid
import ipywidgets as widgets
from google.colab import output
from IPython.display import HTML, clear_output


#@markdown <font size='5'>← Mount / Unmount rclone VFS Drive</font>
store_cache_on = "DISK" #@param ["RAM", "DISK"]
rclone_conf_content = open('/content/rclone.conf').read()
available_remote = re.findall(r"^\[(.+)\]$", rclone_conf_content, re.M)
mount_name = widgets.Dropdown(options=available_remote)

if store_cache_on == 'RAM':
    cache_path = '/dev/shm'
elif store_cache_on == 'DISK':
    os.makedirs('/tmp', exist_ok=True)
    cache_path = '/tmp'


class MakeButton(object):
    def __init__(self, title, callback, style):
        self._title = title
        self._callback = callback
        self._style = style

    def _repr_html_(self):
        callback_id = 'button-' + str(uuid.uuid4())

        output.register_callback(callback_id, self._callback)

        if self._style != "":
            style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
        else:
            style_html = "p-Widget jupyter-widgets jupyter-button widget-button"

        template = """<button class="{style_html}" id="{callback_id}">{title}</button>
            <script>
                document.querySelector("#{callback_id}").onclick = (e) => {{
                    google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
                    e.preventDefault();
                }};
            </script>"""
        html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)

        return html


def run_shell(args, *, output=False, shell=False, cd=None):
    if not shell:
        if output:
            proc = subprocess.Popen(shlex.split(args), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=cd)

            while True:
                output = proc.stdout.readline()

                if output == b"" and proc.poll() is not None:
                    return
                if output:
                    print(output.decode("utf-8").strip())

        return subprocess.run(shlex.split(args), cwd=cd).returncode
    else:
        if output:
            return (subprocess.run(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=cd).stdout.decode("utf-8").strip())

        return subprocess.run(args, shell=True, cwd=cd).returncode


def mount_command():
    mount_path = f"/content/drive/rclone/{mount_name.value}"
    os.makedirs(mount_path, exist_ok=True)

    cmd = rf"rclone mount {mount_name.value}: {mount_path}" \
        rf" --config /content/rclone.conf" \
        ' --user-agent "Mozilla"' \
        ' --buffer-size 256M' \
        ' --transfers 10' \
        ' --vfs-cache-mode full' \
        ' --vfs-cache-max-age 0h0m1s' \
        ' --vfs-cache-poll-interval 0m1s' \
        f' --cache-dir {cache_path}' \
        ' --allow-other' \
        ' --daemon'

    if run_shell(cmd, shell=True) == 0:
        print_html(f'✅ {mount_name.value} has been mounted to {mount_path}', '#4CAF50')
    else:
        print_html(f'❌ Unable to mount {mount_name.value}', '#F44336')


def unmount_command():
    mount_path = f"/content/drive/rclone/{mount_name.value}"

    try:
        subprocess.run(f'fusermount -uz {mount_path}', shell=True, check=True)
    except:
        print_html(f'❌ {mount_name.value} is not mounted', '#F44336')
    else:
        subprocess.run(f'rm -r {mount_path}', shell=True)
        print_html(f'✅ {mount_name.value} has been unmounted', '#4CAF50')


def main_widget():
    clear_output(wait=True)

    display(widgets.HBox(
        [widgets.VBox(
            [widgets.HTML(
                '''
                    <h3 style="font-family: Trebuchet MS; font-size: 18px; font-weight: bold; color: #3F51B5; margin-top: 0px;">Choose which drive to mount / unmount:</h3>
                '''
            ), mount_name]
        )]
    ))
  
    display(HTML("<br>"), MakeButton("Mount", mount_command, "primary"), MakeButton("Unmount", unmount_command, "danger"), HTML("<br>"))


def print_html(text='', color=''):
    display(HTML(f'<p style="font-size: 18px; font-weight: bold; color: {color};">{text}</p>'))


def main():
    mount_path = '/content/drive/rclone'

    if not pathlib.Path(mount_path).exists():
        pathlib.Path(mount_path).mkdir(parents=True, exist_ok=True)

    main_widget()


if __name__ == '__main__':
    main()


In [ ]:
import re
import subprocess
import traceback
from os import chdir
from sys import exit
from IPython.display import HTML


def print_html(text='', color='#FFFFFF'):
    display(HTML(f'<p style="font-size: 18px; font-weight: bold; color: {color};">{text}</p>'))


def main():
    #@markdown <font size='5'>← Run rclone Command</font>
    action = 'copy' #@param ['copy', 'move', 'sync']
    source = '' #@param {type: 'string'}
    destination = '' #@param {type: 'string'}
    #
    #@markdown ---
    extra_arguments = '--check-first' #@param {type: 'string'}
    checkers = 4 #@param {type:"slider", min:1, max:20, step:1}
    transfers = 4 #@param {type:"slider", min:1, max:20, step:1}
    compare_mode = 'size and mod-time' #@param ['size and mod-time', 'size and checksum', 'only mod-time', 'only size', 'only checksum']
    do_not_update_modtime_if_files_are_identical = False #@param {type: 'boolean'}
    skip_all_files_that_exist = True #@param {type: 'boolean'}
    skip_files_that_are_newer_on_the_destination = False #@param {type: 'boolean'}
    google_drive_optimization = False #@param {type: 'boolean'}
    large_amount_of_files_optimization = True #@param {type: 'boolean'}
    simple_output = True #@param {type: 'boolean'}
    dry_run = False #@param {type: 'boolean'}
    #
    #@markdown ---
    sync_mode = 'delete during transfer' #@param ['delete during transfer', 'delete before transfer', 'delete after transfer']
    track_renames = False #@param {type: 'boolean'}
    #
    buffer_size = '--buffer-size=96M'
    rclone_config = '/content/rclone.conf'
    config_param_pattern = re.compile(r'--config="([^"]+)"')
    match = config_param_pattern.search(extra_arguments)

    chdir('/content')

    if compare_mode == 'size and mod-time':
        compare_mode = ''
    elif compare_mode == 'size and checksum':
        compare_mode = '--checksum'
    elif compare_mode == 'only mod-time':
        compare_mode = '--ignore-size'
    elif compare_mode == 'only size':
        compare_mode = '--size-only'
    elif compare_mode == 'only checksum':
        compare_mode = '--checksum --ignore-size'

    if do_not_update_modtime_if_files_are_identical:
        update_modtime = ''
    else:
        update_modtime = '--no-update-modtime'

    if skip_all_files_that_exist:
        skip_exist_files = '--ignore-existing'
    else:
        skip_exist_files = ''

    if skip_files_that_are_newer_on_the_destination:
        skip_newer_files = '--update'
    else:
        skip_newer_files = ''

    if google_drive_optimization:
        gdrive_optimization = '--drive-chunk-size=32M --drive-acknowledge-abuse --drive-keep-revision-forever'
    else:
        gdrive_optimization = ''

    if large_amount_of_files_optimization:
        files_optimization = '--fast-list'
    else:
        files_optimization = ''

    if simple_output:
        console_output = '-v --stats-one-line --stats=5s'
    else:
        console_output = '-v --stats=5s'

    if dry_run:
        test_mode = '--dry-run'
    else:
        test_mode = ''

    if sync_mode == 'delete during transfer':
        sync_mode = '--delete-during'
    elif sync_mode == 'delete before transfer':
        sync_mode = '--delete-before'
    elif sync_mode == 'delete after transfer':
        sync_mode = '--delete-after'
    
    if track_renames:
        track_renames = '--track-renames'
    else:
        track_renames = ''

    if match:
        rclone_config = match.group(1)

    try:
        subprocess.run('rclone --version', shell=True, check=True, capture_output=True)
    except:
        rclone_installed = False
    else:
        rclone_installed = True
    
    if not rclone_installed:
        exit(print_html('❌ rclone is not installed! Please install it first.', '#F44336'))
    if source == '':
        exit(print_html('❌ The source field cannot be empty!', '#F44336'))
    if destination == '':
        exit(print_html('❌ The destination field cannot be empty!', '#F44336'))

    # command = f'rclone {action} {source} {destination} {test_mode} {console_output} --checkers={checkers} --transfers={transfers} {files_optimization} {gdrive_optimization} {skip_newer_files} {skip_exist_files} {update_modtime} --config="{rclone_config}"'
    # with subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True) as process:
    #     for line in process.stdout:
    #         print(line, end='')

    if action == 'copy':
        !rclone {action} {source} {destination} {test_mode} {console_output} --checkers={checkers} --transfers={transfers} {buffer_size} {compare_mode} {files_optimization} {gdrive_optimization} {skip_newer_files} {skip_exist_files} {update_modtime} {extra_arguments} --config="{rclone_config}"
    elif action == 'sync':
        !rclone {action} {source} {destination} {test_mode} {console_output} --checkers={checkers} --transfers={transfers} {buffer_size} {compare_mode} {files_optimization} {gdrive_optimization} {skip_newer_files} {skip_exist_files} {update_modtime} {sync_mode} {track_renames} {extra_arguments} --config="{rclone_config}"
    elif action == 'move':
        !rclone {action} {source} {destination} {test_mode} {console_output} --checkers={checkers} --transfers={transfers} {buffer_size} {compare_mode} {files_optimization} {gdrive_optimization} {skip_newer_files} {skip_exist_files} {update_modtime} --delete-empty-src-dirs {extra_arguments} --config="{rclone_config}"

    print_html('✅ You have reached the end of the line')


if __name__ == '__main__':
    try:
        main()
    except (SystemExit, KeyboardInterrupt):
        pass
    except Exception as error:
        print_html(error)
        traceback.print_exc()


# <font size=6>✦ <a href="https://github.com/AUTOMATIC1111/stable-diffusion-webui">Stable Diffusion WebUI</a>✦ </font>

Rebased on [TheLastBen's Fast Stable Diffusion](https://colab.research.google.com/github/TheLastBen/fast-stable-diffusion/blob/main/fast_stable_diffusion_AUTOMATIC1111.ipynb) and modified

In [ ]:
#@markdown <font size="5">← Install / Update WebUI</font>

import os
import pathlib
import time
from google.colab import drive
from IPython.display import HTML, clear_output
from IPython.utils import capture


version = "latest" #@param ["latest", "rollback"]
commit_hash = 'a9eab236d7e8afa4d6205127904a385b2c43bb24'
#@markdown > Rollback to previous commit in case the latest commit got messed up again.
#
#@markdown ---
install_dir = "" #@param {type:"string"}
#@markdown > Set custom installation path.<br>
#@markdown > You can mount an rclone drive and point it to your rclone remote.

if install_dir != '':
    stable_diffusion_main_dir = f'{install_dir}/stable-diffusion'
else:
    stable_diffusion_main_dir = f'/content/stable-diffusion'
stable_diffusion_webui_dir = f'{stable_diffusion_main_dir}/stable-diffusion-webui'
stable_diffusion_model_dir = f'{stable_diffusion_webui_dir}/models/Stable-diffusion'
stable_diffusion_vae_dir = f'{stable_diffusion_webui_dir}/models/VAE'
stable_diffusion_lora_dir = f'{stable_diffusion_webui_dir}/models/Lora'
stable_diffusion_hypernetwork_dir = f'{stable_diffusion_webui_dir}/models/hypernetworks'
stable_diffusion_upscaler_dir = f'{stable_diffusion_webui_dir}/models/ESRGAN'
stable_diffusion_embedding_dir = f'{stable_diffusion_webui_dir}/embeddings'
stable_diffusion_extension_dir = f'{stable_diffusion_webui_dir}/extensions'


def print_html(text='', color=''):
    display(HTML(f'<p style="font-size: 18px; font-weight: bold; color: {color};">{text}</p>'))


def download(url, output_path='/content'):
    url = url.replace(' ', '%20')

    if 'civitai.com' in url:
        !wget -nc -P '{output_path}' --content-disposition -q --show-progress --progress=bar:force 2>&1 '{url}'
    elif 'huggingface.co' in url:
        if '/blob/' in url:
            url = url.replace('/blob/', '/resolve/')

        !wget -nc -P '{output_path}' -q --show-progress --progress=bar:force 2>&1 '{url}'
    elif 'drive.google.com' in url:
        !gdown '{url}' -O '{output_path}/'
    else:
        !wget -nc -P '{output_path}' -q --show-progress --progress=bar:force 2>&1 '{url}'


os.chdir('/content')
print_html('Installing WebUI...')


with capture.capture_output() as cap:
    pathlib.Path(f'{stable_diffusion_main_dir}').mkdir(parents=True, exist_ok=True)
    os.chdir(f'{stable_diffusion_main_dir}')
    !git clone -q 'https://github.com/AUTOMATIC1111/stable-diffusion-webui'

    pathlib.Path(f'{stable_diffusion_webui_dir}/cache').mkdir(parents=True, exist_ok=True)
    os.environ['TRANSFORMERS_CACHE'] = f'{stable_diffusion_webui_dir}/cache'

    os.chdir(f'{stable_diffusion_webui_dir}')

    time.sleep(1)
    !rm webui.sh

    if version == 'rollback':
        !git reset --hard {commit_hash}
        !pip install gradio==3.16.2 --no-deps -qq
    else:
        !git reset --hard
        !git pull
        !pip install gradio==3.23 --no-deps -qq

    # Pre-downloading some extensions that I think will be good to have
    os.chdir(f'{stable_diffusion_webui_dir}/extensions')
    !git clone -q 'https://github.com/DominikDoom/a1111-sd-webui-tagcomplete' 'a1111-sd-webui-tagcomplete'
    !git clone -q 'https://github.com/Mikubill/sd-webui-controlnet' 'sd-webui-controlnet'
    !git clone -q 'https://github.com/fkunn1326/openpose-editor' 'openpose-editor'
    !git clone -q 'https://github.com/jexom/sd-webui-depth-lib' 'sd-webui-depth-lib'
    # !git clone -q 'https://github.com/imrayya/stable-diffusion-webui-Prompt_Generator' 'prompt-generator'

    # Pre-downloading ControlNet models
    pathlib.Path('sd-webui-controlnet/models').mkdir(parents=True, exist_ok=True)
    !wget -nc -q -P 'sd-webui-controlnet/models' 'https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_canny-fp16.safetensors'
    !wget -nc -q -P 'sd-webui-controlnet/models' 'https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_depth-fp16.safetensors'
    !wget -nc -q -P 'sd-webui-controlnet/models' 'https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_hed-fp16.safetensors'
    !wget -nc -q -P 'sd-webui-controlnet/models' 'https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_mlsd-fp16.safetensors'
    !wget -nc -q -P 'sd-webui-controlnet/models' 'https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_normal-fp16.safetensors'
    !wget -nc -q -P 'sd-webui-controlnet/models' 'https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors'
    !wget -nc -q -P 'sd-webui-controlnet/models' 'https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_scribble-fp16.safetensors'
    !wget -nc -q -P 'sd-webui-controlnet/models' 'https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_seg-fp16.safetensors'

    # Pre-downloading upscaler models
    pathlib.Path(f'{stable_diffusion_webui_dir}/models/ESRGAN').mkdir(parents=True, exist_ok=True)
    pathlib.Path(f'{stable_diffusion_webui_dir}/models/RealESRGAN').mkdir(parents=True, exist_ok=True)
    pathlib.Path(f'{stable_diffusion_webui_dir}/models/SwinIR').mkdir(parents=True, exist_ok=True)
    !wget -nc -q --show-progress --progress=bar:force 2>&1 -P '{stable_diffusion_webui_dir}/models/RealESRGAN' 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth'
    !wget -nc -q --show-progress --progress=bar:force 2>&1 -O '{stable_diffusion_webui_dir}/models/SwinIR/SwinIR_4x.pth' 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'


os.chdir('/content')


clear_output()
print_html('✓ DONE', '#4CAF50')


In [ ]:
#@markdown <font size="5">← Install Requirements</font>

import os
import pathlib
from IPython.display import HTML
from IPython.utils import capture


os.chdir('/content')


print_html('Installing requirements...', '#FFEB3B')
with capture.capture_output() as cap:
    !wget -q -i 'https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt'
    !dpkg -i *.deb
    
    if not pathlib.Path(f'{stable_diffusion_main_dir}/stable-diffusion-stability-ai').exists():
        !tar -C '/content' --zstd -xf 'sd_rep.tar.zst'
        !mv '/content/sd/stablediffusion' '{stable_diffusion_main_dir}/stable-diffusion-stability-ai'
        !rm -rf '/content/sd'
    !tar -C '/' --zstd -xf 'gcolabdeps.tar.zst'
    !rm *.deb | rm *.zst | rm *.txt

    !wget -q -O '/usr/local/lib/python3.9/dist-packages/flax/core/frozen_dict.py' 'https://github.com/TheLastBen/fast-stable-diffusion/raw/main/AUTOMATIC1111_files/frozen_dict.py'
    
    os.environ['LD_PRELOAD'] = 'libtcmalloc.so'
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    os.environ['PYTHONWARNINGS'] = 'ignore'
    
    if version == 'rollback':
        !pip install gradio==3.16.2 --no-deps -qq
    else:
        !pip install gradio==3.23 --no-deps -qq


os.chdir('/content')


clear_output()
print_html('✓ DONE', '#4CAF50')


In [ ]:
import pathlib


#@markdown <font size="5">← Download Models</font>
#
#@markdown Supported sites:
#@markdown - CivitAI (copy the url from the download button on the top right side)
#@markdown - Huggingface
#@markdown - Google Drive (publicly shared url)
#@markdown - Any other direct url to the file
#
#@markdown ---
#@markdown <font size="4">Download Diffusion Model</font>
model_option = 'anything-v4.5-pruned.safetensors' #@param ['DOWNLOAD FROM URL', 'ChilloutMix', 'ChilloutMix-Ni', 'ChilloutMix-Ni-pruned-fp16', 'ChilloutMix-Ni-pruned-fp32', 'ChilloutMix-Ni-pruned-fp16-fix', 'ChilloutMix-Ni-pruned-fp32-fix', 'DosMix', 'RealDosMix', 'anything-v3-full.safetensors', 'anything-v3-fp32-pruned.safetensors', 'anything-v3-fp16-pruned.safetensors', 'anything-v4.5.safetensors', 'anything-v4.5-pruned.safetensors', 'anything-v4.5-pruned-fp32.ckpt', 'anything-v4.5-pruned-fp16.ckpt', 'corneos-7th-heaven-mix', 'Counterfeit-V2.0.ckpt', 'Counterfeit-V2.0.safetensors', 'Counterfeit-V2.0fp16.safetensors', 'Counterfeit-V2.1.safetensors', 'Counterfeit-V2.2.safetensors', 'Counterfeit-V2.5.safetensors', 'Counterfeit-V2.5_fp16.safetensors', 'Counterfeit-V2.5_pruned.safetensors', 'pastelmix.safetensors', 'pastelmix-fp32.safetensors', 'pastelmix-fp16.safetensors', 'pastelmix-better-vae.safetensors', 'pastelmix-better-vae-fp32.safetensors', 'pastelmix-better-vae-fp16.safetensors', 'Meina V4- Baked Vae.safetensors', 'Meina V4.1 - Baked VAE.safetensors', 'Meina V4.safetensors', 'Meina V5 - Baked VAE.safetensors', 'Meina V5.1 - Baked VAE.safetensors', 'Meina V5.1.safetensors', 'Meina V5.safetensors', 'Meina V6 - Baked VAE.safetensors', 'Meina V6.safetensors', 'Meina V7 - Baked VAE ckpt.ckpt', 'Meina V7 - pruned.safetensors', 'Meina V8 baked VAE.safetensors', 'Meina V8 baked ckpt.ckpt', 'Meina V8 pruned.safetensors', 'Meina Version 2.1 .safetensors', 'Meina Version 3.0 - baked VAE.safetensors', 'MeinaHentai - baked VAE.safetensors', 'MeinaPastel - baked VAE.safetensors', 'MeinaPastelV2 - baked VAE- pruned.safetensors']
model_url = '' #@param {type: 'string'}
download_model = True #@param {type:"boolean"}
#
#@markdown ---
#@markdown <font size="4">Download VAE Model</font>
vae_option = 'anything.vae.pt' #@param ['DOWNLOAD FROM URL', 'vae-ft-mse-840000-ema-pruned.safetensors', 'anything.vae.pt', 'Counterfeit-V2.5.vae.pt', 'kl-f8-anime2.ckpt', 'pastel-waifu-diffusion.vae.pt']
vae_url = '' #@param {type: 'string'}
download_vae = True #@param {type:"boolean"}
#
#@markdown ---
#@markdown <font size="4">Download LoRA Model</font>
lora_option = 'DOWNLOAD FROM URL' #@param ['DOWNLOAD FROM URL']
lora_url = 'https://civitai.com/api/download/models/13899' #@param {type: 'string'}
download_lora = False #@param {type:"boolean"}
#
#@markdown ---
#@markdown <font size="4">Download Embedding Model</font>
embedding_option = 'EasyNegative.safetensors' #@param ['DOWNLOAD FROM URL', 'EasyNegative.safetensors']
embedding_url = '' #@param {type: 'string'}
download_embedding = True #@param {type:"boolean"}
#
#@markdown ---
#@markdown <font size="4">Download Upscaler Model</font>
upscaler_option = 'Lollypop' #@param ['DOWNLOAD FROM URL', 'Lollypop']
upscaler_url = '' #@param {type: 'string'}
download_upscaler = True #@param {type:"boolean"}


if download_model:
    if model_option == 'DOWNLOAD FROM URL' and model_url != '':
        url = model_url
    elif model_option == 'ChilloutMix':
        url = 'https://civitai.com/api/download/models/7543'
    elif model_option == 'ChilloutMix-Ni':
        url = 'https://civitai.com/api/download/models/8958'
    elif model_option == 'ChilloutMix-Ni-pruned-fp16':
        url = 'https://civitai.com/api/download/models/9474'
    elif model_option == 'ChilloutMix-Ni-pruned-fp32':
        url = 'https://civitai.com/api/download/models/9475'
    elif model_option == 'ChilloutMix-Ni-pruned-fp16-fix':
        url = 'https://civitai.com/api/download/models/11732'
    elif model_option == 'ChilloutMix-Ni-pruned-fp32-fix':
        url = 'https://civitai.com/api/download/models/11745'
    elif model_option == 'DosMix':
        url = 'https://civitai.com/api/download/models/7328'
    elif model_option == 'RealDosMix':
        url = 'https://civitai.com/api/download/models/8137'
    elif 'anything' in model_option.lower() and 'v3' in model_option.lower():
        url = f'https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/{model_option}'
    elif 'anything' in model_option.lower() and 'v4' in model_option.lower():
        url = f'https://huggingface.co/andite/anything-v4.0/resolve/main/{model_option}'
    elif model_option == 'corneos-7th-heaven-mix':
        url = 'https://civitai.com/api/download/models/6878'
    elif 'counterfeit' in model_option.lower() and 'v2.0' in model_option.lower():
        url = f'https://huggingface.co/gsdf/Counterfeit-V2.0/resolve/main/{model_option}'
    elif 'counterfeit' in model_option.lower() and not 'v2.0' in model_option.lower():
        url = f'https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/{model_option}'
    elif 'pastelmix' in model_option.lower():
        url = f'https://huggingface.co/andite/pastel-mix/resolve/main/{model_option}'
    elif 'meina' in model_option.lower():
        url = f'https://huggingface.co/Meina/MeinaMix/resolve/main/{model_option}'

    pathlib.Path(stable_diffusion_model_dir).mkdir(parents=True, exist_ok=True)

    download(url, stable_diffusion_model_dir)


if download_vae:
    if vae_option == 'DOWNLOAD FROM URL' and vae_url != '':
        url = vae_url
    elif vae_option == 'vae-ft-mse-840000-ema-pruned.safetensors':
        url = 'vae-ft-mse-840000-ema-pruned.safetensors'
    elif vae_option == 'kl-f8-anime2.ckpt':
        url = 'https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt'
    elif vae_option == 'anything.vae.pt':
        url = 'https://huggingface.co/NoCrypt/Anything-v3-0/resolve/main/anything.vae.pt'
    elif vae_option == 'Counterfeit-V2.5.vae.pt':
        url = 'https://huggingface.co/gsdf/Counterfeit-V2.5/resolve/main/Counterfeit-V2.5.vae.pt'
    elif vae_option == 'pastel-waifu-diffusion.vae.pt':
        url = 'https://huggingface.co/andite/pastel-mix/resolve/main/pastel-waifu-diffusion.vae.pt'

    pathlib.Path(stable_diffusion_vae_dir).mkdir(parents=True, exist_ok=True)

    download(url, stable_diffusion_vae_dir)


if download_lora:
    if lora_option == 'DOWNLOAD FROM URL' and lora_url != '':
        url = lora_url
    
    pathlib.Path(stable_diffusion_lora_dir).mkdir(parents=True, exist_ok=True)

    download(url, stable_diffusion_lora_dir)


if download_embedding:
    if embedding_option == 'DOWNLOAD FROM URL' and embedding_url != '':
        url = embedding_url
    elif embedding_option == 'EasyNegative.safetensors':
        url = 'https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.safetensors'
    
    pathlib.Path(stable_diffusion_embedding_dir).mkdir(parents=True, exist_ok=True)

    download(url, stable_diffusion_embedding_dir)


if download_upscaler:
    if upscaler_option == 'DOWNLOAD FROM URL' and upscaler_url != '':
        url = upscaler_url
    elif upscaler_option == 'Lollypop':
        url = 'https://drive.google.com/u/1/uc?id=10h8YXKKOQ61ANnwLjjHqXJdn4SbBuUku&export=download'

    pathlib.Path(stable_diffusion_upscaler_dir).mkdir(parents=True, exist_ok=True)

    download(url, stable_diffusion_upscaler_dir)


print_html('\n✓ DONE', '#4CAF50')


In [ ]:
#@markdown <font size="5">← Keep Colab Alive on Mobile</font></br>
#@markdown Run this cell to keep the tab alive on mobile devices (before launching the WebUI)

%%html
<b>Press play on the music player to keep the tab alive before running the start cell (Uses only 13MB of data)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@markdown <font size="5">← Launch WebUI</font>

import fileinput
import os
import random
import sys
import time
from subprocess import getoutput
from IPython.utils import capture
from pyngrok import ngrok, conf


webui_username = "" #@param {type: 'string'}
webui_password = "" #@param {type: 'string'}
#@markdown > Set an authentification for the WebUI.
#
#@markdown ---
webui_auth = f'--gradio-auth {webui_username}:{webui_password}'
tunneling = "cloudflare" #@param ["cloudflare", "ngrok", "gradio"]
ngrok_token = "" #@param {type: 'string'}
use_free_token = True #@param {type:"boolean"}
#@markdown > If `use_free_token` is checked, it will randomly choose a token from the free_token list.<br>
#@markdown > In case there's an issue with the picked token, re-run the cell to pick a different token.<br>
#@markdown > `These tokens are not mine` and found mostly on github.

ngrok_free_token = [
    '1X7aYWPuFKYzvewLbnNoMo71kZi_2uzbB966Q4TU5cpgNPKhy',
    '1UqHsShi6o3ketf426P5UtVdTfs_5XFD6sFRMkryka8fAbLd3',
    '7LE18LK8zeaDYeybp5fKP_6GNG1oHEfhTnQE7s9qpw',
    '1Qe1IeySOQWSTnpQ3eFfr8j7Oi5_2zhanqnpZwHBhsfANd6yf',
    '7pWLVhS1gxiMAQdaFeYJy_31krnw9drNLLJftaNSFnm',
    '1XJNNnG8kZsPjjFmLsYNWCC0gIo_7VpBhwTcvhiuK4o2G2jbt',
    '1XzP70k7YVrg7MMaHQWPks0Q8Za_7y6b1mTDJDmJWcuqt5qTp',
    '1Y14GB7E4acXxWYnVTiBejgnLuV_853z7mAgaTJxE9KY3HnCW',
    '1XkoKNLcyiPECcQfGUjrTVzN64P_7tv2YgC4DSnazyVtpCpHm',
    '1Xc7z0uHxDoI9Ah06EQKgH61zoP_6WTPXDGvjFmcp2o7gNmqa',
    '3c4WZaxPbjeRwRibY5opU_2N4TTRKaDubtEWMeKkFXn',
    '3fW4eXHdUN3ziCBXcahZ_3tnDdaTyLw8tKzJtKZkLp',
    '3CqeFZQht43cG5Z2YKfyv_6aKTrgrbo1HtyRi78hRKK',
    '1RCQwctVjSz8AIzHO6S55jm8XB8_5N6PqyZVnoN7mUVqF1yvT',
    '1XTxsRKP8XyxvaJigX9XFXU2FvK_4dqzLxNRJHBz8A3aoPC85',
    '3Y8YSw6bvC9CsbYeRczmt_8akMuLYA3bAUshP1NCMnW',
    '1XSYq8gmxzNgMlYQzERmC50uBot_6qURZnj43KsYF2GWaUamm'
]

if use_free_token:
    ngrok_token = random.choice(ngrok_free_token)

if webui_username == '' or webui_password == '':
    webui_auth = ''


os.chdir('/content')
with capture.capture_output() as cap:
    os.chdir(f'{stable_diffusion_webui_dir}/modules')
    if version == 'rollback':
        !wget -q -O paths.py https://github.com/TheLastBen/fast-stable-diffusion/raw/5632d2ef7fffd940976538d270854ec4faf26855/AUTOMATIC1111_files/paths.py
        !wget -q -O extras.py https://github.com/AUTOMATIC1111/stable-diffusion-webui/raw/a9eab236d7e8afa4d6205127904a385b2c43bb24/modules/extras.py
        !wget -q -O sd_models.py https://github.com/AUTOMATIC1111/stable-diffusion-webui/raw/a9eab236d7e8afa4d6205127904a385b2c43bb24/modules/sd_models.py
        !wget -q -O /usr/local/lib/python3.9/dist-packages/gradio/blocks.py https://github.com/TheLastBen/fast-stable-diffusion/raw/7ff88eaa1fb4997bacd9845bd487f9a14335d625/AUTOMATIC1111_files/blocks.py 
    else:
        !wget -q -O paths.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/paths.py
        !wget -q -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/extras.py
        !wget -q -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/sd_models.py
        !wget -q -O /usr/local/lib/python3.9/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
    
    os.chdir(f'{stable_diffusion_webui_dir}')
    !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" {stable_diffusion_webui_dir}/modules/sd_models.py
    !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {stable_diffusion_webui_dir}/webui.py
    !sed -i "s@map_location='cpu'@map_location='cuda'@" {stable_diffusion_webui_dir}/modules/extras.py
    !sed -i 's@/content/gdrive/MyDrive/sd/stablediffusion@{stable_diffusion_main_dir}/stable-diffusion-stability-ai@' {stable_diffusion_webui_dir}/modules/paths.py


if tunneling == 'cloudflare':
    tunneling = ''

    with capture.capture_output() as cap:
        !pkill cloudflared
        time.sleep(4)

        !nohup cloudflared tunnel --url http://localhost:7860 > /content/srv.txt 2>&1 &
        time.sleep(4)

        !grep -o 'https[^[:space:]]*\.trycloudflare.com' /content/srv.txt >/content/srvr.txt
        time.sleep(2)

        srv = getoutput('cat /content/srvr.txt')

        for line in fileinput.input('/usr/local/lib/python3.9/dist-packages/gradio/blocks.py', inplace=True):
            if line.strip().startswith('self.server_name ='):
                line = f'            self.server_name = "{srv[8:]}"\n'
            if line.strip().startswith('self.protocol = "https"'):
                line = '            self.protocol = "https"\n'
            if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
                line = ''
            if line.strip().startswith('else "http"'):
                line = ''
            sys.stdout.write(line)
                
        !rm /content/srv.txt /content/srvr.txt
elif tunneling == 'ngrok':
    if ngrok_token != '':
        tunneling = ''

        ngrok.kill()
        srv = ngrok.connect(7860, pyngrok_config=conf.PyngrokConfig(auth_token=ngrok_token), bind_tls=True).public_url

        for line in fileinput.input('/usr/local/lib/python3.9/dist-packages/gradio/blocks.py', inplace=True):
            if line.strip().startswith('self.server_name ='):
                line = f'            self.server_name = "{srv[8:]}"\n'

            if line.strip().startswith('self.protocol = "https"'):
                line = '            self.protocol = "https"\n'

            if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
                line = ''

            if line.strip().startswith('else "http"'):
                line = ''

            sys.stdout.write(line)
    else:
        tunneling = '--share'
else:
    tunneling = '--share'


!python 'webui.py' {tunneling} {webui_auth} --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers --disable-console-progressbars
